In [1]:
import numpy as np
import pandas as pd
import utm
import math
import pickle
import os
import sys
sys.path.append('./src')
import time
import datetime
from collections import defaultdict
import matplotlib
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import copy

In [12]:
import hmm, uni_hmm
from data_cleaning import noise, load, counter

# Cell Tower PDF

In [4]:
load = reload(load)
file_2g = './data/jiading/data_2g.csv'
gongcan_2g = './data/jiading/2g_gongcan.csv'
final_gongcan_2g = './data/jiading/final_2g_gongcan.csv'

In [5]:
db, db_gps, towers = load.load_2g(file_2g, gongcan_2g, merge_tower=False, neighbor=False, with_rssi=False, radio_angle=False, context=False)
print 'len(db):', len(db), 'len(towers):', len(towers)

Unique Cell Tower:  81
Duplicated cell towers: [[66, 67, 68], [35, 36], [53, 54], [69, 70, 71], [40, 41], [56, 57], [61, 62, 63]]
Totally duplicate: 86
len(db): 72 len(towers): 91


In [6]:
noise = reload(noise)
dbs = noise.clean_db(db, max_dist=100, min_dist=100, min_len=10, max_again=10, debug=True)
dbs_gps = noise.clean_db_gps(db_gps, max_dist=100, min_dist=100, min_len=10, max_again=10, debug=False)
len(dbs), len(dbs_gps)

Traj ID=0
0->1:286m	
discard[0:0]=0m	keep[1:24]as id=0	
Traj ID=1
7->8:510m	
discard[0:7]=0m	keep[9:157]as id=1	
Traj ID=2
1->2:330m	
discard[0:1]=0m	keep[3:77]as id=2	
Traj ID=3

keep[1:33]as id=3	
Traj ID=4
0->1:394m	
discard[0:0]=0m	keep[2:44]as id=4	
Traj ID=5
1->2:118m	
discard[0:1]=0m	keep[3:48]as id=5	
Traj ID=6
1->2:522m	112->113:752m	133->134:116m	
discard[0:1]=0m	trimed[0,89] keep[2:23]as id=6	discard[113:133]=33m	discard[134:135]=0m	
Traj ID=7
9->10:101m	
discard[0:9]=0m	keep[10:112]as id=7	
Traj ID=8

keep[0:373]as id=8	
Traj ID=9
1->2:947m	
discard[0:1]=0m	keep[2:341]as id=9	
Traj ID=10

keep[0:420]as id=10	
Traj ID=11

keep[1:295]as id=11	
Traj ID=12

keep[1:422]as id=12	
Traj ID=13
35->36:838m	
discard[0:35]=0m	keep[37:1015]as id=13	
Traj ID=14

keep[1:826]as id=14	
Traj ID=15
42->43:667m	
keep[5:42]as id=15	keep[43:114]as id=16	
Traj ID=16
2->3:482m	345->346:714m	
discard[0:2]=0m	keep[6:345]as id=17	keep[346:853]as id=18	
Traj ID=17

trimed[1,12] keep[1:234]as id=19	
Tr

(74, 74)

In [7]:
load = reload(load)
noise = reload(noise)
match_res, time_set = load.load_matching('./data/jiading/matching_out', len(dbs), 50)
dbs = noise.reclean(dbs, time_set, debug=True)
dbs_gps = noise.reclean(dbs_gps, time_set)

traj id= 1 150 140
traj id= 13 980 976
traj id= 22 190 167
traj id= 23 216 198
traj id= 25 427 420
traj id= 26 185 149
traj id= 41 181 173
traj id= 44 136 131
traj id= 50 219 218
traj id= 51 322 316
traj id= 63 34 32
traj id= 67 89 87
traj id= 72 50 44


In [10]:
hmm = reload(hmm)
# train:(origin idx, origin feature)
# train, test = hmm.k_split(dbs, k=4)
train, test, gps3 = hmm.k_g_split(dbs, k=4, gpsize=0.0)

In [14]:
uni_hmm = reload(uni_hmm)
bounding_box = (328000, 331000, 3462000, 3465000)
side = 30
sag = uni_hmm.Sag(side, bounding_box)

In [21]:
tower_param = dict()
for val in towers.itervalues():
    index, lat, lng = val[:3]
    x, y, _, _ = utm.from_latlon(lat, lng)
    tower_param[index] = sag.utm2index(x, y)

In [24]:
tower_dict = dict()
for tr_id, traj in train.iteritems():
    for old_idx, point in traj:
        x, y = point[:2]
        tower = point[2]
        cx, cy = sag.utm2index(x, y)
        tx, ty = tower_param[tower]
        dist = math.floor(math.sqrt((tx-cx)**2 + (ty-cy)**2))
        if not tower_dict.has_key(tower):
            tower_dict[tower] = dict()
        if not tower_dict[tower].has_key(dist):
            tower_dict[tower][dist] = 0
        tower_dict[tower][dist] += 1

In [27]:
tower_dict[2]

{24.0: 0.00906344410876133,
 25.0: 0.0030211480362537764,
 26.0: 0.006042296072507553,
 27.0: 0.01812688821752266,
 28.0: 0.006042296072507553,
 29.0: 0.03323262839879154,
 30.0: 0.07854984894259819,
 31.0: 0.13595166163141995,
 32.0: 0.08459214501510574,
 33.0: 0.04833836858006042,
 34.0: 0.06948640483383686,
 35.0: 0.015105740181268883,
 36.0: 0.05740181268882175,
 37.0: 0.006042296072507553,
 38.0: 0.12084592145015106,
 39.0: 0.08459214501510574,
 40.0: 0.0634441087613293,
 41.0: 0.06042296072507553,
 42.0: 0.012084592145015106,
 43.0: 0.00906344410876133,
 44.0: 0.021148036253776436,
 46.0: 0.01812688821752266,
 47.0: 0.012084592145015106,
 48.0: 0.00906344410876133,
 50.0: 0.015105740181268883,
 52.0: 0.0030211480362537764}

In [26]:
for tower_id, cell_dict in tower_dict.iteritems():
    total = sum(cell_dict.itervalues())
    for dist, cnt in cell_dict.iteritems():
        cell_dict[dist] = cnt * 1.0 / total

In [ ]:
tower_id = 2
cell_dict = tower_dict[tower_id]
